<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
from src.db import get_dataframe, save_to_duckdb

# Add description here
#
# *Note:* You can open this file as a notebook (JupyterLab: right-click on it in the side bar -> Open With -> Notebook)

In [2]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [3]:
# If this task has dependencies, list them them here
# (e.g. upstream = ['some_task']), otherwise leave as None.
upstream = ['s01_get']

# This is a placeholder, leave it as None
product = None
table_name = None
transformed_table_name = None

In [4]:
# Parameters
table_name = "house_prices"
transformed_table_name = "house_prices_transformed"
upstream = {"s01_get": {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/products/01-extract-pipeline.ipynb", "csv_file": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/data/house_prices_raw.csv", "database": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/data/house_prices.duckdb"}}
product = {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/products/04-transform.ipynb"}


In [5]:
# your code here...
db_file = upstream['s01_get']['database']
df = get_dataframe(duckdb_file=db_file, table_name=table_name)

In [6]:
from sklearn.preprocessing import StandardScaler

features =  df.columns # ["col1", "col2", "col3", "col4"]
autoscaler = StandardScaler()
df[features] = autoscaler.fit_transform(df[features])

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
save_to_duckdb(df=df, table_name=transformed_table_name, db_path=upstream['database'])

KeyError: 'database'